In [1]:
# general
import os
import numpy as np
import pandas as pd

# plots
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
sns.set()
from IPython.display import Image

# predictions
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
drivers = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/drivers.csv")
driver_standings = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/driver_standings.csv", sep = ",")
driver_results = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/results.csv")
pit_stops = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/pit_stops.csv")
qualifying = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/qualifying.csv")
races = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/races.csv")
circuits = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/circuits.csv")

lap_times = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/lap_times.csv")
status = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/status.csv")
constructors = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/constructors.csv")
constructor_standings = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/constructor_standings.csv")
constructor_results = pd.read_csv("https://raw.githubusercontent.com/the-mcw/ML1/main/data/constructor_results.csv")

In [3]:
races_1 = races.loc[(races['year'] >= 2011) & (races['year'] <= 2020)].copy().iloc[:,0:6]
del races_1['name']
races_2 = pd.merge(races_1,circuits[['circuitId','country']], on = 'circuitId')
races_2.date = pd.to_datetime(races_2.date)
races_2.head()

,raceId,year,round,circuitId,date,country
0,841,2011,1,1,2011-03-27,Australia
1,860,2012,1,1,2012-03-18,Australia
2,880,2013,1,1,2013-03-17,Australia
3,900,2014,1,1,2014-03-16,Australia
4,926,2015,1,1,2015-03-15,Australia


In [4]:
#race_1 = pd.merge(races, pit_stops, on='raceId', how='outer')
#race_1 = race_1.rename(columns={'time_y':'TimeOfPitStop'})
#Filtering Race seasons 2011 - 2020
#race_2011_2020 = race_1.loc[(race_1['year'] >= 2011) & (race_1['year'] <= 2020)].copy()
#Convert duration attribute to numeric
#race_2011_2020["duration"] = pd.to_numeric(race_2011_2020["duration"], errors="coerce")
#pit_stops.head()
pits = pd.pivot_table(pit_stops, values='milliseconds', index= ['raceId','driverId'], columns='stop').reset_index()
pits['mean_stop'] = pits.mean(axis=1)
pits['stops'] = 6-pits.isna().sum(axis=1)
pits =pits[['raceId','driverId','mean_stop','stops']]
pits.columns.name = None
pits = pits.merge(races_2,on='raceId').drop(columns='circuitId')
pits

,raceId,driverId,mean_stop,stops,year,round,date,country
0,841,1,11817.000000,2,2011,1,2011-03-27,Australia
1,841,2,12233.750000,2,2011,1,2011-03-27,Australia
2,841,3,8186.666667,1,2011,1,2011-03-27,Australia
3,841,4,14602.000000,3,2011,1,2011-03-27,Australia
4,841,5,8570.333333,1,2011,1,2011-03-27,Australia
...,...,...,...,...,...,...,...,...
3889,1047,846,7911.000000,1,2020,17,2020-12-13,UAE
3890,1047,847,7993.000000,1,2020,17,2020-12-13,UAE
3891,1047,848,7801.000000,1,2020,17,2020-12-13,UAE
3892,1047,849,11409.250000,2,2020,17,2020-12-13,UAE


In [5]:
drivers = drivers[['driverId','driverRef','dob','nationality']]
drivers['dob'] = pd.to_datetime(drivers.dob)
drivers.info()
#driver_standings = driver_standings.drop(columns=['driverStandingsId','positionText'])
driver_standings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     857 non-null    int64         
 1   driverRef    857 non-null    object        
 2   dob          857 non-null    datetime64[ns]
 3   nationality  857 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 26.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33902 entries, 0 to 33901
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   driverStandingsId  33902 non-null  int64  
 1   raceId             33902 non-null  int64  
 2   driverId           33902 non-null  int64  
 3   points             33902 non-null  float64
 4   position           33902 non-null  int64  
 5   positionText       33902 non-null  object 
 6   wins               33902 non-null  

In [6]:
drivers_1 = pd.merge(drivers, driver_standings, on='driverId', how='outer')
drivers_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33909 entries, 0 to 33908
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   driverId           33909 non-null  int64         
 1   driverRef          33909 non-null  object        
 2   dob                33909 non-null  datetime64[ns]
 3   nationality        33909 non-null  object        
 4   driverStandingsId  33902 non-null  float64       
 5   raceId             33902 non-null  float64       
 6   points             33902 non-null  float64       
 7   position           33902 non-null  float64       
 8   positionText       33902 non-null  object        
 9   wins               33902 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 2.8+ MB


In [7]:
driver_results.position = driver_results.position.replace('\\N',np.NaN).astype(np.float64)
driver_results.rank = driver_results['rank'].replace('\\N',np.NaN).astype(np.float64)
driver_results = driver_results[['raceId','driverId','constructorId','grid','positionOrder','points','laps','rank','fastestLapTime','fastestLapSpeed','statusId']]
driver_results

,raceId,driverId,constructorId,grid,positionOrder,points,laps,rank,fastestLapTime,fastestLapSpeed,statusId
0,18,1,1,1,1,10.0,58,2,1:27.452,218.300,1
1,18,2,2,5,2,8.0,58,3,1:27.739,217.586,1
2,18,3,3,7,3,6.0,58,5,1:28.090,216.719,1
3,18,4,4,11,4,5.0,58,7,1:28.603,215.464,1
4,18,5,1,3,5,4.0,58,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,854,210,12,16,0.0,57,12,1:29.833,211.632,11
25836,1096,825,210,16,17,0.0,57,20,1:31.158,208.556,11
25837,1096,1,131,5,18,0.0,55,11,1:29.788,211.738,9
25838,1096,849,3,20,19,0.0,55,14,1:30.309,210.517,130


In [8]:
drivers_2 = pd.merge(drivers_1, driver_results, on=('driverId','raceId'), how='outer', suffixes=('Season','Race'))
drivers_2

,driverId,driverRef,dob,nationality,driverStandingsId,raceId,pointsSeason,position,positionText,wins,constructorId,grid,positionOrder,pointsRace,laps,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,hamilton,1985-01-07,British,1.0,18.0,10.0,1.0,1,1.0,1.0,1.0,1.0,10.0,58.0,2,1:27.452,218.300,1.0
1,1,hamilton,1985-01-07,British,9.0,19.0,14.0,1.0,1,1.0,1.0,9.0,5.0,4.0,56.0,3,1:35.462,209.033,1.0
2,1,hamilton,1985-01-07,British,27.0,20.0,14.0,3.0,3,1.0,1.0,3.0,13.0,0.0,56.0,19,1:35.520,203.969,11.0
3,1,hamilton,1985-01-07,British,48.0,21.0,20.0,2.0,2,1.0,1.0,5.0,3.0,6.0,66.0,3,1:22.017,204.323,1.0
4,1,hamilton,1985-01-07,British,69.0,22.0,28.0,3.0,3,1.0,1.0,3.0,2.0,8.0,58.0,2,1:26.529,222.085,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34464,486,NaN,NaT,NaN,NaN,742.0,NaN,NaN,NaN,NaN,102.0,20.0,19.0,0.0,56.0,\N,\N,\N,2.0
34465,155,NaN,NaT,NaN,NaN,841.0,NaN,NaN,NaN,NaN,15.0,9.0,21.0,0.0,58.0,9,1:30.384,211.218,2.0
34466,815,NaN,NaT,NaN,NaN,841.0,NaN,NaN,NaN,NaN,15.0,13.0,22.0,0.0,58.0,6,1:29.962,212.209,2.0
34467,24,NaN,NaT,NaN,NaN,841.0,NaN,NaN,NaN,NaN,164.0,0.0,23.0,0.0,0.0,0,\N,\N,81.0


In [9]:
drivers_3 = pd.merge(drivers_2,pits, on=('raceId','driverId'))
drivers_3['rank'] = drivers_3['rank'].replace('\\N',np.NaN).astype(np.float64)
drivers_3['podium'] = drivers_3.positionOrder<4

drivers_3.date = pd.to_datetime(drivers_3.date) 
drivers_3['age'] = (drivers_3.date-drivers_3.dob)/np.timedelta64(1,'Y')
del drivers_3['dob']
drivers_3['fastestLapSpeed'] = drivers_3['fastestLapSpeed'].replace('\\N',np.NaN).astype(np.float64)

In [10]:
print(drivers_3.head())
drivers_3.info()

   driverId driverRef nationality  driverStandingsId  raceId  pointsSeason  \
0         1  hamilton     British            68157.0   969.0          18.0   
1         1  hamilton     British            64385.0   842.0          22.0   
2         1  hamilton     British            64691.0   841.0          18.0   
3         1  hamilton     British            64409.0   843.0          47.0   
4         1  hamilton     British            64433.0   844.0          59.0   

   position positionText  wins  constructorId  grid  positionOrder  \
0       2.0            2   0.0          131.0   1.0            2.0   
1       3.0            3   0.0            1.0   2.0            8.0   
2       2.0            2   0.0            1.0   2.0            2.0   
3       2.0            2   1.0            1.0   3.0            1.0   
4       2.0            2   1.0            1.0   4.0            4.0   

   pointsRace  laps  rank fastestLapTime  fastestLapSpeed  statusId  \
0        18.0  57.0   6.0       1:27.03

In [13]:
drivers_4 = drivers_3.merge(qualifying.drop(columns=['qualifyId','number']),on=['raceId','driverId','constructorId'],
                            suffixes=('Season','Qualify'))
drivers_4['q1'] = drivers_4.q1.replace('\\N',np.NaN)
drivers_4['q2'] = drivers_4.q2.replace('\\N',np.NaN)
drivers_4['q3'] = drivers_4.q3.replace('\\N',np.NaN)
drivers_4

,driverId,driverRef,nationality,driverStandingsId,raceId,pointsSeason,positionSeason,positionText,wins,constructorId,grid,positionOrder,pointsRace,laps,rank,fastestLapTime,fastestLapSpeed,statusId,mean_stop,stops,year,round,date,country,podium,age,positionQualify,q1,q2,q3
0,1,hamilton,British,68157.0,969.0,18.0,2.0,2,0.0,131.0,1.0,2.0,18.0,57.0,6.0,1:27.033,219.351,1.0,7559.666667,1,2017,1,2017-03-26,Australia,True,32.214214,1,1:24.191,1:23.251,1:22.188
1,1,hamilton,British,64385.0,842.0,22.0,3.0,3,0.0,1.0,2.0,8.0,4.0,56.0,5.0,1:41.512,196.575,1.0,15642.333333,4,2011,2,2011-04-10,Malaysia,False,26.253790,2,1:36.861,1:35.852,1:34.974
2,1,hamilton,British,64691.0,841.0,18.0,2.0,2,0.0,1.0,2.0,2.0,18.0,58.0,8.0,1:30.314,211.382,1.0,11817.000000,2,2011,1,2011-03-27,Australia,True,26.215460,2,1:25.384,1:24.595,1:24.307
3,1,hamilton,British,64409.0,843.0,47.0,2.0,2,1.0,1.0,3.0,1.0,25.0,56.0,2.0,1:40.415,195.424,1.0,12564.400000,3,2011,3,2011-04-17,China,True,26.272956,3,1:36.091,1:34.486,1:34.463
4,1,hamilton,British,64433.0,844.0,59.0,2.0,2,1.0,1.0,4.0,4.0,12.0,58.0,4.0,1:30.108,213.264,1.0,16747.000000,4,2011,4,2011-05-08,Turkey,False,26.330452,4,1:27.091,1:26.066,1:25.595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,850,pietro_fittipaldi,Brazilian,70163.0,1046.0,0.0,23.0,23,0.0,210.0,20.0,17.0,0.0,87.0,17.0,0:57.742,220.892,1.0,16350.000000,3,2020,16,2020-12-06,Bahrain,False,24.449510,20,0:55.426,NaN,NaN
3862,850,pietro_fittipaldi,Brazilian,70209.0,1047.0,0.0,23.0,23,0.0,210.0,17.0,19.0,0.0,53.0,8.0,1:41.707,196.588,12.0,15988.800000,3,2020,17,2020-12-13,UAE,False,24.468675,19,1:38.173,NaN,NaN
3863,851,aitken,British,70162.0,1046.0,0.0,22.0,22,0.0,3.0,17.0,16.0,0.0,87.0,15.0,0:57.392,222.240,1.0,18495.200000,3,2020,16,2020-12-06,Bahrain,False,25.205172,18,0:54.892,NaN,NaN
3864,155,NaN,NaN,NaN,841.0,NaN,NaN,NaN,NaN,15.0,9.0,21.0,0.0,58.0,9.0,1:30.384,211.218,2.0,12313.000000,2,2011,1,2011-03-27,Australia,False,NaN,9,1:25.717,1:25.405,1:25.626
